Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [17]:
batch_size = 128
image_pixels = image_size * image_size
hidden_nodes = 1024
import math

def get_nn_logits(images, valid=False):
  with tf.name_scope('hidden'):
    weights = tf.Variable(
        tf.truncated_normal([image_pixels, hidden_nodes], stddev=1.0 / math.sqrt(float(image_pixels))))
    biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden = tf.nn.relu(tf.matmul(images, weights) + biases)
    
  # train computation.
  with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
          tf.truncated_normal([hidden_nodes, num_labels],
                              stddev=1.0 / math.sqrt(float(hidden_nodes))))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden, weights) + biases
  return logits

graph = tf.Graph()
with graph.as_default():
  # placeholders
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  logits = get_nn_logits(tf_dataset)
  
  # loss
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_labels))
  loss = tf.nn.l2_loss(loss)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)

    
num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
#       print "step %d, training accuracy %g"%(step, accuracy.eval(feed_dict=feed_dict))

      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      valid_offset = (step * batch_size) % (valid_labels.shape[0] - batch_size)
      print "Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={
                tf_dataset: valid_dataset[valid_offset:(valid_offset + batch_size), :],
                tf_labels: valid_labels[valid_offset:(valid_offset + batch_size), :]
            }), valid_labels[valid_offset:(valid_offset + batch_size), :])
  print "Test accuracy: %.1f%%" % accuracy(prediction.eval(
        feed_dict={tf_dataset: test_dataset, tf_labels: test_labels}), test_labels)


Initialized
Minibatch loss at step 0 : 2.68337
Minibatch accuracy: 14.8%
Validation accuracy: 26.6%
Minibatch loss at step 500 : 0.103767
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 1000 : 0.125659
Minibatch accuracy: 83.6%
Validation accuracy: 88.3%
Minibatch loss at step 1500 : 0.0583824
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 2000 : 0.0549991
Minibatch accuracy: 89.1%
Validation accuracy: 85.9%
Minibatch loss at step 2500 : 0.0819669
Minibatch accuracy: 86.7%
Validation accuracy: 89.8%
Minibatch loss at step 3000 : 0.142756
Minibatch accuracy: 86.7%
Validation accuracy: 88.3%
Test accuracy: 94.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
num_steps = 3
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
#       print "step %d, training accuracy %g"%(step, accuracy.eval(feed_dict=feed_dict))

      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      valid_offset = (step * batch_size) % (valid_labels.shape[0] - batch_size)
      print "Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={
                tf_dataset: valid_dataset[valid_offset:(valid_offset + batch_size), :],
                tf_labels: valid_labels[valid_offset:(valid_offset + batch_size), :]
            }), valid_labels[valid_offset:(valid_offset + batch_size), :])
  print "Test accuracy: %.1f%%" % accuracy(prediction.eval(
        feed_dict={tf_dataset: test_dataset, tf_labels: test_labels}), test_labels)

Initialized
Minibatch loss at step 0 : 2.74249
Minibatch accuracy: 11.7%
Validation accuracy: 31.2%
Test accuracy: 26.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
batch_size = 128
image_pixels = image_size * image_size
hidden_nodes = 1024
import math

graph = tf.Graph()
with graph.as_default():
  # placeholders
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  keep_prob = tf.placeholder(tf.float32)
  logits = get_nn_logits(tf_dataset, keep_prob)

  with tf.name_scope('hidden'):
    h_weights = tf.Variable(
        tf.truncated_normal([image_pixels, hidden_nodes], stddev=1.0 / math.sqrt(float(image_pixels))))
    h_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden = tf.nn.relu(tf.matmul(tf_dataset, h_weights) + h_biases)
    hidden_dropped = tf.nn.dropout(hidden, keep_prob)
    
  # train computation.
  with tf.name_scope('softmax_linear'):
    s_weights = tf.Variable(
          tf.truncated_normal([hidden_nodes, num_labels],
                              stddev=1.0 / math.sqrt(float(hidden_nodes))))
    s_biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden_dropped, s_weights) + s_biases

  # loss
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_labels))
  loss += 5e-4 * (tf.nn.l2_loss(h_weights) + tf.nn.l2_loss(h_biases) +
                 tf.nn.l2_loss(s_weights) + tf.nn.l2_loss(s_biases))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)

    
num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
#       print "step %d, training accuracy %g"%(step, accuracy.eval(feed_dict=feed_dict))

      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      valid_offset = (step * batch_size) % (valid_labels.shape[0] - batch_size)
      print "Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={
                tf_dataset: valid_dataset[valid_offset:(valid_offset + batch_size), :],
                tf_labels: valid_labels[valid_offset:(valid_offset + batch_size), :],
                keep_prob: 1.0
            }), valid_labels[valid_offset:(valid_offset + batch_size), :])
  print "Test accuracy: %.1f%%" % accuracy(prediction.eval(
        feed_dict={tf_dataset: test_dataset, tf_labels: test_labels, keep_prob: 1.0}), test_labels)


Initialized
Minibatch loss at step 0 : 2.5378
Minibatch accuracy: 10.9%
Validation accuracy: 16.4%
Minibatch loss at step 500 : 0.795646
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Minibatch loss at step 1000 : 0.853294
Minibatch accuracy: 82.8%
Validation accuracy: 85.9%
Minibatch loss at step 1500 : 0.731839
Minibatch accuracy: 80.5%
Validation accuracy: 79.7%
Minibatch loss at step 2000 : 0.656331
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500 : 0.817715
Minibatch accuracy: 85.2%
Validation accuracy: 89.1%
Minibatch loss at step 3000 : 0.844429
Minibatch accuracy: 82.0%
Validation accuracy: 85.9%
Test accuracy: 91.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
